In [ ]:
#hide
# default_exp core
from nbdev.showdoc import *

# kicad-helpers

> Scripts, templates, and examples for managing KiCad projects.

In [ ]:
#export
import glob
import os

import git

In [ ]:
#export
def get_project_root():
    # Find the current projects' root directory
    git_repo = git.Repo(".", search_parent_directories=True)
    return git_repo.git.rev_parse("--show-toplevel")

In [ ]:
#export
def get_project_name():
    # This is based on the name of the KiCad `*.pro` file.
    root = get_project_root()
    return os.path.splitext(os.path.split(glob.glob(os.path.join(root, "*.pro"))[0])[1])[0]

In [ ]:
#export
def say_hello(to):
    "Say hello to somebody"
    return f'Hello {to}!'

In [ ]:
say_hello("Sylvain")

'Hello Sylvain!'

In [ ]:
# Examples can output plots, images, etc, and they'll all appear in your docs, e.g.:

from IPython.display import display,SVG

display(SVG('<svg height="100"><circle cx="50" cy="50" r="40"/></svg>'))

In [ ]:
# You can also include tests:

assert say_hello("Jeremy")=="Hello Jeremy!"